In [ ]:
MODEL_PATH = "models/nepali/ne_NP-chitwan-medium.onnx"

In [7]:
GOOGLE_MODEL_PATH = "models/nepali/ne_NP-google-medium.onnx"

In [11]:
import wave
import numpy as np
import sounddevice as sd
from piper.voice import PiperVoice



# Load once, reuse many times
voice = PiperVoice.load(GOOGLE_MODEL_PATH)

# --- Option A: Save to WAV file (easiest to inspect quality) ---
def speak_to_file(text: str, output_path: str):
    with wave.open(output_path, "w") as wav_file:
        voice.synthesize(text, wav_file)
    print(f"Saved to {output_path}")

# --- Option B: Play directly through speakers (streaming) ---
def speak_live(text: str):
    stream = sd.OutputStream(
        samplerate=voice.config.sample_rate,
        channels=1,
        dtype="int16"
    )
    stream.start()
    # synthesize() now yields AudioChunk objects
    for chunk in voice.synthesize(text):
        stream.write(np.frombuffer(chunk.audio_int16_bytes, dtype=np.int16))
    stream.stop()
    stream.close()

# --- Option C: Command line (quickest sanity check) ---
# echo "नमस्ते, तपाईलाई के सहायता चाहियो?" | piper \
#   --model models/nepali/ne_NP-chitwan-medium.onnx \
#   --output_file output.wav


# ---- Test with your actual booking dialog sentences ----
test_sentences = [
    # Pure Nepali
    "नमस्ते, तपाईलाई के सहायता चाहियो?",
    
    # Nepalinglish (this will stress-test phonemization)
    "तपाईको मन्डे को लागि बुकिङ कन्फर्म भयो है।",
    
    # Numbers and time (common in booking flows)
    "तपाईको appointment दुई घण्टाको लागि छ।",
    
    # Location reference
    "Kupondole ma तपाईको नजिकको provider छ। कान देउता मन्दिरको अगडि।",
]

for sentence in test_sentences:
    print(f"\nSynthesizing: {sentence}")
    #speak_to_file(sentence, f"output_{test_sentences.index(sentence)}.wav")
    speak_live(sentence)


Synthesizing: नमस्ते, तपाईलाई के सहायता चाहियो?

Synthesizing: तपाईको मन्डे को लागि बुकिङ कन्फर्म भयो है।

Synthesizing: तपाईको appointment दुई घण्टाको लागि छ।

Synthesizing: Kupondole ma तपाईको नजिकको provider छ। कान देउता मन्दिरको अगडि।


ALSA lib pcm.c:8740:(snd_pcm_recover) underrun occurred
